In [1]:
!pip install -U folium 

  Using cached https://files.pythonhosted.org/packages/49/df/50aa1999ab9bde74656c2919d9c0c085fd2b3775fd3eca826012bef76d8c/requests-2.18.4-py2.py3-none-any.whl
Requirement already up-to-date: jinja2 in /srv/app/venv/lib/python3.6/site-packages (from folium)
Requirement already up-to-date: six in /srv/app/venv/lib/python3.6/site-packages (from folium)
  Using cached https://files.pythonhosted.org/packages/c3/fe/c140768eaf9d14ef6e844d0e9679b7085d87ceeaa578020fd6fed679d74c/branca-0.2.0-py3-none-any.whl
Requirement already up-to-date: chardet<3.1.0,>=3.0.2 in /srv/app/venv/lib/python3.6/site-packages (from requests->folium)
Requirement already up-to-date: urllib3<1.23,>=1.21.1 in /srv/app/venv/lib/python3.6/site-packages (from requests->folium)
Requirement already up-to-date: certifi>=2017.4.17 in /srv/app/venv/lib/python3.6/site-packages (from requests->folium)
Requirement already up-to-date: idna<2.7,>=2.5 in /srv/app/venv/lib/python3.6/site-packages (from requests->folium)
Requirement al

In [2]:
from datascience import *
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import folium
import json
import os
from branca.colormap import linear
import branca.colormap

In [3]:
race = Table.read_table('ACS_16_1YR_B02001_with_ann.csv').drop('HD02_VD01')
variables = list(race.labels)[4:]
#race.select(variables)
race

GEO.id         | GEO.id2 | GEO.display-label      | HD01_VD01        | HD01_VD02                      | HD02_VD02                             | HD01_VD03                                          | HD02_VD03                                                 | HD01_VD04                                                  | HD02_VD04                                                    | HD01_VD05                      | HD02_VD05                             | HD01_VD06                                                    | HD02_VD06                                                    | HD01_VD07                                | HD02_VD07                                       | HD01_VD08                             | HD02_VD08                                    | HD01_VD09                                                    | HD02_VD09                                                    | HD01_VD10                                                    | HD02_VD10
Id             | Id2     | Geography              | Estimate; Total: | Estimate; Total: - White alone | Margin of Error; Total: - White alone | Estimate; Total: - Black or African American alone | Margin of Error; Total: - Black or African American alone | Estimate; Total: - American Indian and Alaska Native alone | Margin of Error; Total: - American Indian and Alaska Nat ... | Estimate; Total: - Asian alone | Margin of Error; Total: - Asian alone | Estimate; Total: - Native Hawaiian and Other Pacific Isl ... | Margin of Error; Total: - Native Hawaiian and Other Paci ... | Estimate; Total: - Some other race alone | Margin of Error; Total: - Some other race alone | Estimate; Total: - Two or more races: | Margin of Error; Total: - Two or more races: | Estimate; Total: - Two or more races: - Two races includ ... | Margin of Error; Total: - Two or more races: - Two races ... | Estimate; Total: - Two or more races: - Two races exclud ... | Margin of Error; Total: - Two or more races: - Two races ...
0500000US48005 | 48005   | Angelina County, Texas | 87791            | 67968                          | 2438                                  | 13621                                              | 564                                                       | 1342                                                       | 1626                                                         | 1007                           | 220                                   | 47                                                           | 93                                                           | 3217                                     | 1511                                            | 589                                   | 537                                          | 70                                                           | 117                                                          | 519                                                          | 513
0500000US48021 | 48021   | Bastrop County, Texas  | 82733            | 58948                          | 4129                                  | 5621                                               | 1621                                                      | 431                                                        | 691                                                          | 1293                           | 1358                                  | 0                                                            | 220                                                          | 14508                                    | 4146                                            | 1932                                  | 1390                                         | 935                                                          | 852                                                          | 997                                                          | 1116
0500000US48027 | 48027   | Bell County, Texas     | 340411           | 217524                         | 4802                

In [4]:
county_pop = race.select(2, 'HD01_VD01').relabeled(1, 'pop estimate').take(np.arange(1, race.num_rows))
county_pop

GEO.display-label      | pop estimate
Angelina County, Texas | 87791
Bastrop County, Texas  | 82733
Bell County, Texas     | 340411
Bexar County, Texas    | 1928680
Bowie County, Texas    | 93860
Brazoria County, Texas | 354195
Brazos County, Texas   | 220417
Cameron County, Texas  | 422135
Collin County, Texas   | 939585
Comal County, Texas    | 134788
... (40 rows omitted)

In [5]:
race_df = race.to_df().iloc[1:]
race_df.head()
county_name = race_df[['GEO.display-label']]

In [41]:
a = race_df[variables].astype('float')
percent_df = a.div(race_df['HD01_VD01'].astype('float'), axis=0)
#df[['B','C']].div(df.A, axis=0)
percent_df['GEO.display-label'] = race_df['GEO.display-label']
percent_df.head()

HD01_VD02  HD02_VD02  HD01_VD03  HD02_VD03  HD01_VD04  HD02_VD04  \
1   0.774202   0.027771   0.155153   0.006424   0.015286   0.018521   
2   0.712509   0.049908   0.067941   0.019593   0.005210   0.008352   
3   0.639004   0.014106   0.242683   0.011859   0.005714   0.003599   
4   0.788502   0.007060   0.078809   0.002368   0.007394   0.002637   
5   0.702163   0.008225   0.243682   0.011411   0.003175   0.003931   

   HD01_VD05  HD02_VD05  HD01_VD06  HD02_VD06  HD01_VD07  HD02_VD07  \
1   0.011470   0.002506   0.000535   0.001059   0.036644   0.017211   
2   0.015629   0.016414   0.000000   0.002659   0.175359   0.050113   
3   0.028483   0.003707   0.007388   0.000861   0.020516   0.006281   
4   0.029063   0.000922   0.000665   0.000312   0.067215   0.006554   
5   0.013478   0.001758   0.003345   0.004880   0.007245   0.003590   

   HD01_VD08  HD02_VD08  HD01_VD09  HD02_VD09  HD01_VD10  HD02_VD10  \
1   0.006709   0.006117   0.000797   0.001333   0.005912   0.005843   
2   0.023352   0.016801   0.011301   0.010298   0.012051   0.013489   
3   0.056211   0.008337   0.003049   0.001533   0.053162   0.008540   
4   0.028352   0.002557   0.006943   0.001308   0.021408   0.002113   
5   0.026912   0.012178   0.002078   0.001662   0.024835   0.012103   

        GEO.display-label  
1  Angelina County, Texas  
2   Bastrop County, Texas  
3      Bell County, Texas  
4     Bexar County, Texas  
5     Bowie County, Texas

In [42]:
race_percent = Table.from_df(percent_df)
for i in variables:
    race_percent = race_percent.relabeled(i, race.column(i).item(0))
race_percent

Estimate; Total: - White alone | Margin of Error; Total: - White alone | Estimate; Total: - Black or African American alone | Margin of Error; Total: - Black or African American alone | Estimate; Total: - American Indian and Alaska Native alone | Margin of Error; Total: - American Indian and Alaska Native alone | Estimate; Total: - Asian alone | Margin of Error; Total: - Asian alone | Estimate; Total: - Native Hawaiian and Other Pacific Islander alone | Margin of Error; Total: - Native Hawaiian and Other Pacific Islander alone | Estimate; Total: - Some other race alone | Margin of Error; Total: - Some other race alone | Estimate; Total: - Two or more races: | Margin of Error; Total: - Two or more races: | Estimate; Total: - Two or more races: - Two races including Some other race | Margin of Error; Total: - Two or more races: - Two races including Some other race | Estimate; Total: - Two or more races: - Two races excluding Some other race, and three or more races | Margin of Error; Total: - Two or more races: - Two races excluding Some other race, and three or more races | GEO.display-label
0.774202                       | 0.0277705                             | 0.155153                                           | 0.00642435                                                | 0.0152863                                                  | 0.0185213                                                         | 0.0114704                      | 0.00250595                            | 0.000535362                                                         | 0.00105933                                                                 | 0.0366438                                | 0.0172113                                       | 0.00670912                            | 0.0061168                                    | 0.000797348                                                                 | 0.00133271                                                                         | 0.00591177                                                                                           | 0.00584342                                                                                                  | Angelina County, Texas
0.712509                       | 0.0499075                             | 0.0679415                                          | 0.0195931                                                 | 0.00520953                                                 | 0.00835217                                                        | 0.0156286                      | 0.0164142                             | 0                                                                   | 0.00265916                                                                 | 0.175359                                 | 0.050113                                        | 0.0233522                             | 0.016801                                     | 0.0113014                                                                   | 0.0102982                                                                          | 0.0120508                                                                                            | 0.0134892                                                                                                   | Bastrop County, Texas
0.639004                       | 0.0141065                             | 0.242683                                           | 0.0118592                                                 | 0.00571368                                                 | 0.00359859                                                        | 0.0284832                      | 0.00370728                            | 0.00738813                                                          | 0.000860724                                                                | 0.0205164                                | 0.00628064                                      | 0.0562115                           

In [10]:
mean_income = Table.read_table('ACS_16_1YR_income_with_ann.csv')
mean_income = mean_income.select('GEO.display-label', 'HC02_EST_VC02').relabeled(
    0, 'County').relabeled(1, 'mean income').take(np.arange(1, mean_income.num_rows))
mean_income

County                 | mean income
Angelina County, Texas | 55000
Bastrop County, Texas  | 72650
Bell County, Texas     | 68060
Bexar County, Texas    | 72419
Bowie County, Texas    | 67583
Brazoria County, Texas | 90604
Brazos County, Texas   | 71130
Cameron County, Texas  | 51206
Collin County, Texas   | 113597
Comal County, Texas    | 99722
... (43 rows omitted)

In [48]:
latino = Table.read_table('ACS_16_1YR_latinopop_with_ann.csv')
latino = latino.select(2, 3, 'HD01_VD12').relabeled(
    0, 'County').relabeled(1, 'total pop').relabeled(2,'latino pop').take(
    np.arange(1, latino.num_rows))
latino

County                 | total pop | latino pop
Bell County, Texas     | 340411    | 82567
Bexar County, Texas    | 1928680   | 1155996
Brazoria County, Texas | 354195    | 106243
Brazos County, Texas   | 220417    | 55379
Cameron County, Texas  | 422135    | 377343
Collin County, Texas   | 939585    | 142209
Coryell County, Texas  | 74686     | 13425
Dallas County, Texas   | 2574984   | 1026256
Denton County, Texas   | 806180    | 154800
Ector County, Texas    | 157462    | 94056
... (23 rows omitted)

### Caseload
https://tidc.tamu.edu/public.net/Reports/AttorneyCaseLoad.aspx
data from Texas Indigent Defense Commission 
statewide attorney caseload report 2016

In [11]:
caseload= Table.read_table('CaseLoadReport.csv')
caseload

County  | Bar #    | Attorney Name            | Juvenile Cases Paid | Capital Murder Cases Paid | Adult Felony Cases Paid | Adult Misdemeanor Cases Paid | Juvenile Appeals Cases Paid | Felony Appeals Cases Paid | Misdemeanor Appeals Cases Paid | Total Cases | Total Paid  | Adult Time Percent | Juvenile Time Percent | FY Reported
Crane   | 786000   | ABALOS, RICHARD C.       | 3                   | nan                       | nan                     | 4                            | nan                         | nan                       | nan                            | 7           | $2,450.00   | nan                | nan                   | 2016
Ector   | 786000   | ABALOS, RICHARD C.       | 7                   | nan                       | nan                     | 1                            | nan                         | nan                       | nan                            | 8           | $2,800.00   | nan                | nan                   | 2016
Liberty | 24072501 | ABBATE, THOMAS JOSEPH    | nan                 | nan                       | nan                     | nan                          | nan                         | 1                         | nan                            | 1           | $1,875.00   | 15 %               | nan                   | 2016
Harris  | 785645   | ABBEY, KIMBERLY DAWN     | 0                   | 1                         | 166                     | 483                          | 0                           | 0                         | 0                              | 650         | $163,555.50 | 70 %               | nan                   | 2016
Ellis   | 786949   | ABBOTT, DAVID KURT       | 9                   | 0                         | 17                      | 39                           | 0                           | 0                         | 0                              | 65          | $29,487.50  | 5 %                | 3 %                   | 2016
Randall | 24006725 | ABBOTT, JAMES L.         | nan                 | nan                       | 20                      | 6                            | nan                         | 3                         | nan                            | 29          | $17,450.00  | 10 %               | nan                   | 2016
Potter  | 24006725 | ABBOTT, JAMES L.         | nan                 | nan                       | 49                      | 4                            | nan                         | 2                         | nan                            | 55          | $28,850.00  | 30 %               | nan                   | 2016
Dallas  | 783502   | ABDAL-KHALLAQ, HAMIDA A. | 0                   | 0                         | 76                      | 41                           | 0                           | 0                         | 0                              | 117         | $46,075.00  | 40 %               | nan                   | 2016
Tarrant | 783502   | ABDAL-KHALLAQ, HAMIDA A. | 9                   | nan                       | 61                      | 22                           | nan                         | nan                       | nan                            | 92          | $47,350.00  | 35 %               | 10 %                  | 2016
Denton  | 24043516 | ABEL, JAMES CHRISTOPHER  | nan                 | nan                       | 55                      | 51                           | nan                         | 6                         | 1                              | 113         | $81,772.43  | 40 %               | nan                   | 2016
... (9283 rows omitted)

In [22]:
county_cases = caseload.group('County')
cases_df = county_cases.to_df()
cases_df['County'] = cases_df['County'].astype(str) + ' County, TX'
county_cases = Table.from_df(cases_df)
cases_df.head()

County  count
0  Anderson County, TX     16
1   Andrews County, TX     14
2  Angelina County, TX     23
3   Aransas County, TX     39
4    Archer County, TX     21

In [13]:
pop_df = county_pop.to_df().head()
pop_df['GEO.display-label'] = pop_df['GEO.display-label'].map(lambda x: x.replace('Texas','TX'))
pop = Table.from_df(pop_df)

In [14]:
pop

GEO.display-label   | pop estimate
Angelina County, TX | 87791
Bastrop County, TX  | 82733
Bell County, TX     | 340411
Bexar County, TX    | 1928680
Bowie County, TX    | 93860

In [15]:
county_cases.show(5)

In [16]:
result = pop.join( "GEO.display-label", county_cases,  'County').relabeled('GEO.display-label', 'County')
result_df = result.to_df()
result_df['cases per capita'] = result_df['count'] / result_df['pop estimate'].astype(float)
result_df

County pop estimate  count  cases per capita
0  Angelina County, TX        87791     23          0.000262
1   Bastrop County, TX        82733     21          0.000254
2      Bell County, TX       340411     90          0.000264
3     Bexar County, TX      1928680    617          0.000320
4     Bowie County, TX        93860     20          0.000213

### MAPPING

In [17]:
texas = os.path.join( 'texas','texas.json')
geo_json_data = json.load(open(texas))
tx_coords = [31.9686, -99.9018]

In [18]:
m = folium.Map(tx_coords, zoom_start=5)
m

In [19]:
folium.GeoJson(geo_json_data
).add_to(m)

m

In [43]:
### CASELOAD MAP: 

In [44]:
colormap1 = linear.BuGn.scale(
    cases_df['count'].min(),
    cases_df['count'].max())

colormap1

In [45]:
cases_dict = result_df.set_index('County')['count']
cases_dict['King County, TX'] = 0

In [46]:

c = folium.Map(tx_coords, zoom_start=4)

folium.GeoJson(
    geo_json_data,
    name='cases',
    style_function=lambda feature: {
        'fillColor': colormap1(0) if (not(feature['properties']['name'] in cases_dict)) else colormap1(cases_dict[feature['properties']['name']]),
        'color': 'black',
        'weight': 1,
        'dashArray': '5, 5',
        'fillOpacity': 0.9,
    }
    
).add_to(c)

folium.LayerControl().add_to(c)

c

In [47]:
### CASELOAD PER CAPITA MAP:

In [59]:
colormap2 = linear.BuGn.scale(
    result_df['cases per capita'].min(),
    result_df['cases per capita'].max())

colormap2

In [49]:
per_capita_dict = result_df.set_index('County')['cases per capita']
per_capita_dict['King County, TX'] = 0
per_capita_dict['Taylor County, TX'] = 0

In [50]:
m = folium.Map(tx_coords, zoom_start=5)

folium.GeoJson(
    geo_json_data,
    name='cases per capita',
    style_function=lambda feature: {
        'fillColor': colormap2(0) if (not(feature['properties']['name'] in per_capita_dict)) else colormap2(per_capita_dict[feature['properties']['name']]),
        'color': 'black',
        'weight': 1,
        'dashArray': '5, 5',
        'fillOpacity': 0.9,
    }
    
).add_to(m)

folium.LayerControl().add_to(m)

m

In [51]:
### RACE MAP
percent_df.head()

HD01_VD02  HD02_VD02  HD01_VD03  HD02_VD03  HD01_VD04  HD02_VD04  \
1   0.774202   0.027771   0.155153   0.006424   0.015286   0.018521   
2   0.712509   0.049908   0.067941   0.019593   0.005210   0.008352   
3   0.639004   0.014106   0.242683   0.011859   0.005714   0.003599   
4   0.788502   0.007060   0.078809   0.002368   0.007394   0.002637   
5   0.702163   0.008225   0.243682   0.011411   0.003175   0.003931   

   HD01_VD05  HD02_VD05  HD01_VD06  HD02_VD06  HD01_VD07  HD02_VD07  \
1   0.011470   0.002506   0.000535   0.001059   0.036644   0.017211   
2   0.015629   0.016414   0.000000   0.002659   0.175359   0.050113   
3   0.028483   0.003707   0.007388   0.000861   0.020516   0.006281   
4   0.029063   0.000922   0.000665   0.000312   0.067215   0.006554   
5   0.013478   0.001758   0.003345   0.004880   0.007245   0.003590   

   HD01_VD08  HD02_VD08  HD01_VD09  HD02_VD09  HD01_VD10  HD02_VD10  \
1   0.006709   0.006117   0.000797   0.001333   0.005912   0.005843   
2   0.023352   0.016801   0.011301   0.010298   0.012051   0.013489   
3   0.056211   0.008337   0.003049   0.001533   0.053162   0.008540   
4   0.028352   0.002557   0.006943   0.001308   0.021408   0.002113   
5   0.026912   0.012178   0.002078   0.001662   0.024835   0.012103   

        GEO.display-label  
1  Angelina County, Texas  
2   Bastrop County, Texas  
3      Bell County, Texas  
4     Bexar County, Texas  
5     Bowie County, Texas

In [52]:
percent_df['GEO.display-label'] = percent_df[
                                'GEO.display-label'].map(lambda x: x.replace('Texas','TX'))
percent_df

HD01_VD02  HD02_VD02  HD01_VD03  HD02_VD03  HD01_VD04  HD02_VD04  \
1    0.774202   0.027771   0.155153   0.006424   0.015286   0.018521   
2    0.712509   0.049908   0.067941   0.019593   0.005210   0.008352   
3    0.639004   0.014106   0.242683   0.011859   0.005714   0.003599   
4    0.788502   0.007060   0.078809   0.002368   0.007394   0.002637   
5    0.702163   0.008225   0.243682   0.011411   0.003175   0.003931   
6    0.747594   0.010195   0.129581   0.006070   0.010141   0.007821   
7    0.758825   0.017104   0.102714   0.006565   0.004841   0.005771   
8    0.950262   0.007505   0.005074   0.000993   0.000808   0.000888   
9    0.704097   0.005620   0.093409   0.002357   0.003813   0.000999   
10   0.894048   0.020091   0.023006   0.001795   0.000883   0.001054   
11   0.721407   0.015612   0.105923   0.018959   0.012854   0.010229   
12   0.617941   0.005996   0.226163   0.001845   0.002527   0.000645   
13   0.743345   0.007426   0.092038   0.003928   0.004952   0.001182   
14   0.852555   0.021402   0.048539   0.007380   0.006789   0.006421   
15   0.837595   0.011703   0.098072   0.007353   0.006552   0.004570   
16   0.804790   0.012711   0.035844   0.002813   0.006397   0.001733   
17   0.524705   0.009750   0.205218   0.004175   0.004151   0.002016   
18   0.783757   0.010391   0.131903   0.005907   0.007759   0.003421   
19   0.865739   0.018575   0.043101   0.008648   0.010543   0.003618   
20   0.745460   0.010982   0.208437   0.009762   0.004315   0.002400   
21   0.685885   0.028216   0.070917   0.008940   0.001204   0.001114   
22   0.627677   0.005141   0.190484   0.001800   0.003307   0.000660   
23   0.901100   0.010451   0.042192   0.004098   0.001541   0.000988   
24   0.899914   0.007622   0.059761   0.009211   0.011614   0.007735   
25   0.843922   0.017399   0.003438   0.002182   0.002298   0.001499   
26   0.778741   0.017345   0.083412   0.003606   0.012870   0.001542   
27   0.590002   0.005662   0.337916   0.003118   0.003204   0.001677   
28   0.914114   0.011521   0.033318   0.004985   0.007221   0.006210   
29   0.834804   0.013764   0.107588   0.005163   0.002738   0.002231   
30   0.849860   0.022361   0.102981   0.009485   0.004932   0.002950   
31   0.818218   0.012387   0.070932   0.004985   0.012869   0.006472   
32   0.771141   0.010168   0.147144   0.003114   0.004437   0.002146   
33   0.855768   0.015526   0.070679   0.011079   0.002098   0.003346   
34   0.863681   0.009034   0.054646   0.006704   0.006548   0.004941   
35   0.903039   0.005695   0.040886   0.003077   0.004115   0.003155   
36   0.882927   0.003566   0.088567   0.003048   0.004778   0.004284   
37   0.938791   0.008668   0.015181   0.003971   0.007602   0.002982   
38   0.784718   0.012472   0.100644   0.010990   0.002499   0.001788   
39   0.891540   0.014596   0.026641   0.006166   0.007532   0.003494   
40   0.859786   0.018994   0.063164   0.005565   0.000575   0.000873   
41   0.938393   0.013628   0.015505   0.003562   0.003799   0.003340   
42   0.758063   0.011536   0.173385   0.004971   0.004847   0.001540   
43   0.678143   0.006837   0.158847   0.002547   0.004701   0.001013   
44   0.775281   0.017695   0.078266   0.007808   0.008306   0.002109   
45   0.893324   0.014571   0.050150   0.006470   0.004274   0.002813   
46   0.744576   0.008144   0.082921   0.002371   0.004370   0.001787   
47   0.873793   0.018926   0.067429   0.009171   0.003753   0.004791   
48   0.953609   0.013253   0.004355   0.001099   0.001320   0.001254   
49   0.789181   0.014738   0.096399   0.009193   0.010831   0.006705   
50   0.797773   0.008651   0.059762   0.004023   0.003174   0.001570   

    HD01_VD05  HD02_VD05  HD01_VD06  HD02_VD06  HD01_VD07  HD02_VD07  \
1    0.011470   0.002506   0.000535   0.001059   0.036644   0.017211   
2    0.015629   0.016414   0.000000   0.002659   0.175359   0.050113   
3    0.028483   0.003707   0.007388   0.000861   0.020516   0.006281   
4    0.029063   0.000922   0.000665   0.000

In [53]:
colormap = linear.BuGn.scale(
    round(percent_df['HD01_VD03'].min(), 2),
    round(percent_df['HD01_VD03'].max(), 2))

colormap

In [54]:
african_american_dict = percent_df.set_index('GEO.display-label')['HD01_VD03']
african_american_dict['San Jacinto County, TX'] = 0
african_american_dict['Freestone County, TX'] = 0
african_american_dict['Montague County, TX'] = 0
african_american_dict['Throckmorton County, TX'] = 0
african_american_dict['Jim Wells County, TX'] = 0
african_american_dict['Maverick County, TX'] = 0
african_american_dict['Kimble County, TX'] = 0
african_american_dict['Gaines County, TX'] = 0
african_american_dict['Marion County, TX'] = 0
african_american_dict['Jack County, TX'] = 0
african_american_dict['Reagan County, TX'] = 0
african_american_dict['Pecos County, TX'] = 0
african_american_dict['Shackelford County, TX'] = 0
african_american_dict['Martin County, TX'] = 0
african_american_dict['Crane County, TX'] = 0
african_american_dict['Titus County, TX'] = 0
african_american_dict['Panola County, TX'] = 0
african_american_dict['Knox County, TX'] = 0


In [55]:
c =folium.Map(tx_coords, zoom_start=5)

In [56]:

folium.GeoJson(
    geo_json_data,
    name='Race = African American',
    style_function=lambda feature: {
        'fillColor': colormap(0) if (not(feature['properties']['name'] in african_american_dict)) else colormap(african_american_dict[feature['properties']['name']])
                        , 
        'color': 'black',
        'weight': 1,
        'dashArray': '5, 5',
        'fillOpacity': 0.9,
    }
    
).add_to(c)

folium.LayerControl().add_to(c)

c